In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
df = pd.read_csv('~/Downloads/UCI_Credit_Card.csv')
df.shape

(30000, 24)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         30000 non-null  int64  
 1   LIMIT_BAL  30000 non-null  float64
 2   SEX        30000 non-null  int64  
 3   EDUCATION  30000 non-null  int64  
 4   MARRIAGE   30000 non-null  int64  
 5   AGE        30000 non-null  int64  
 6   PAY_0      30000 non-null  int64  
 7   PAY_2      30000 non-null  int64  
 8   PAY_3      30000 non-null  int64  
 9   PAY_4      30000 non-null  int64  
 10  PAY_5      30000 non-null  int64  
 11  PAY_6      30000 non-null  int64  
 12  BILL_AMT1  30000 non-null  float64
 13  BILL_AMT2  30000 non-null  float64
 14  BILL_AMT3  30000 non-null  float64
 15  BILL_AMT4  30000 non-null  float64
 16  BILL_AMT5  30000 non-null  float64
 17  BILL_AMT6  30000 non-null  float64
 18  PAY_AMT1   30000 non-null  float64
 19  PAY_AMT2   30000 non-null  float64
 20  PAY_AM

In [5]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0


In [6]:
# 1. Check value distributions for categorical columns
print("="*50)
print("CATEGORICAL DISTRIBUTIONS")
print("="*50)

# SEX: 1=male, 2=female
print("SEX values:")
print(df['SEX'].value_counts(normalize=True))
print("\nUnique SEX values:", df['SEX'].unique())

# EDUCATION: 1=graduate, 2=university, 3=high school, 4+=other
print("\nEDUCATION values:")
print(df['EDUCATION'].value_counts().sort_index())
print("Unique values:", df['EDUCATION'].unique())

# MARRIAGE: 1=married, 2=single, 3=other
print("\nMARRIAGE values:")
print(df['MARRIAGE'].value_counts().sort_index())
print("Unique values:", df['MARRIAGE'].unique())

CATEGORICAL DISTRIBUTIONS
SEX values:
SEX
2    0.603733
1    0.396267
Name: proportion, dtype: float64

Unique SEX values: [2 1]

EDUCATION values:
EDUCATION
0       14
1    10585
2    14030
3     4917
4      123
5      280
6       51
Name: count, dtype: int64
Unique values: [2 1 3 5 4 6 0]

MARRIAGE values:
MARRIAGE
0       54
1    13659
2    15964
3      323
Name: count, dtype: int64
Unique values: [1 2 3 0]


In [7]:
# Further investigation of EDUCATION and MARRIAGE
print("="*50)
print("DATA QUALITY INVESTIGATION")
print("="*50)

# Check if 0s appear together (might indicate missing records)
print("Records with EDUCATION=0:")
print(df[df['EDUCATION'] == 0][['ID', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE']].head())

print("\nRecords with MARRIAGE=0:")
print(df[df['MARRIAGE'] == 0][['ID', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE']].head())

# Check if these overlap
edu_0 = set(df[df['EDUCATION'] == 0]['ID'])
mar_0 = set(df[df['MARRIAGE'] == 0]['ID'])
print(f"\nOverlap between EDUCATION=0 and MARRIAGE=0: {len(edu_0.intersection(mar_0))} records")

DATA QUALITY INVESTIGATION
Records with EDUCATION=0:
          ID  SEX  EDUCATION  MARRIAGE  AGE
3769    3770    2          0         2   38
5945    5946    1          0         2   39
6876    6877    1          0         2   30
14631  14632    2          0         2   53
15107  15108    1          0         2   45

Records with MARRIAGE=0:
        ID  SEX  EDUCATION  MARRIAGE  AGE
218    219    2          3         0   31
809    810    2          2         0   37
820    821    2          3         0   51
1019  1020    2          3         0   45
1443  1444    1          3         0   51

Overlap between EDUCATION=0 and MARRIAGE=0: 0 records


In [8]:
# Check payment status columns
print("="*50)
print("PAYMENT STATUS DISTRIBUTIONS (CLEAN)")
print("="*50)

pay_cols = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
for col in pay_cols:
    unique_vals = df[col].unique().tolist()  
    unique_vals.sort()
    print(f"\n{col} unique values: {unique_vals}")

PAYMENT STATUS DISTRIBUTIONS (CLEAN)

PAY_0 unique values: [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]

PAY_2 unique values: [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]

PAY_3 unique values: [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]

PAY_4 unique values: [-2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8]

PAY_5 unique values: [-2, -1, 0, 2, 3, 4, 5, 6, 7, 8]

PAY_6 unique values: [-2, -1, 0, 2, 3, 4, 5, 6, 7, 8]


In [9]:
# checking for -2 usage
no_usage = df[df['PAY_0'] == -2]
print(no_usage[['BILL_AMT1', 'PAY_AMT1']].describe())
# If bills and payments are mostly 0, then -2 likely = no usage

           BILL_AMT1       PAY_AMT1
count    2759.000000    2759.000000
mean     8793.052555    6477.920261
std     28224.565390   20877.338447
min    -15308.000000       0.000000
25%         0.000000       0.000000
50%      1179.000000    1131.000000
75%      5819.000000    5000.000000
max    478030.000000  368199.000000


In [14]:
# Compare -2 vs -1 customers
print("Status -2 vs -1 comparison:")
status_neg1 = df[df['PAY_0'] == -1]
status_neg2 = df[df['PAY_0'] == -2]

print(f"\n-2 Status: {len(status_neg2)} customers")
print(f"  Avg balance: ${status_neg2['BILL_AMT1'].mean():.0f}")
print(f"  Avg payment: ${status_neg2['PAY_AMT1'].mean():.0f}")

print(f"\n-1 Status: {len(status_neg1)} customers")  
print(f"  Avg balance: ${status_neg1['BILL_AMT1'].mean():.0f}")
print(f"  Avg payment: ${status_neg1['PAY_AMT1'].mean():.0f}")

# Check if -2 customers have negative bills (credit balance)
print(f"\n-2 customers with negative bills: {(status_neg2['BILL_AMT1'] < 0).sum()}")

Status -2 vs -1 comparison:

-2 Status: 2759 customers
  Avg balance: $8793
  Avg payment: $6478

-1 Status: 5686 customers
  Avg balance: $10485
  Avg payment: $7395

-2 customers with negative bills: 209


In [10]:
# Check for logical inconsistencies
print("="*50)
print("LOGICAL CONSISTENCY CHECKS")
print("="*50)

# Check: If someone paid in full (PAY_X = -1), their next bill should reflect that
print("Checking payment logic:")
for i in range(1, 6):
    paid_full = df[df[f'PAY_{i+1 if i==1 else i}'] == -1]
    print(f"\nPeople who paid in full in month {i}:")
    print(f"  Count: {len(paid_full)}")
    print(f"  Their next month bill > 0: {(paid_full[f'BILL_AMT{i}'] > 0).sum()}")

LOGICAL CONSISTENCY CHECKS
Checking payment logic:

People who paid in full in month 1:
  Count: 6050
  Their next month bill > 0: 5451

People who paid in full in month 2:
  Count: 6050
  Their next month bill > 0: 5349

People who paid in full in month 3:
  Count: 5938
  Their next month bill > 0: 5373

People who paid in full in month 4:
  Count: 5687
  Their next month bill > 0: 5108

People who paid in full in month 5:
  Count: 5539
  Their next month bill > 0: 4940


In [11]:
# Final summary of data quality issues found
print("="*50)
print("DATA QUALITY SUMMARY")
print("="*50)

issues = []
if df['ID'].duplicated().sum() > 0:
    issues.append(f"- {df['ID'].duplicated().sum()} duplicate IDs")
if (df['EDUCATION'] == 0).sum() > 0:
    issues.append(f"- {(df['EDUCATION'] == 0).sum()} records with EDUCATION=0")
if (df['MARRIAGE'] == 0).sum() > 0:
    issues.append(f"- {(df['MARRIAGE'] == 0).sum()} records with MARRIAGE=0")

print("Issues to handle in staging:")
for issue in issues:
    print(issue)

DATA QUALITY SUMMARY
Issues to handle in staging:
- 14 records with EDUCATION=0
- 54 records with MARRIAGE=0


In [12]:
# Final EDA Summary
print("="*50)
print("EDA FINDINGS SUMMARY")
print("="*50)

print("\n1. DATA SHAPE:")
print(f"   - {df.shape[0]:,} customers, {df.shape[1]} features")
print(f"   - No missing values")
print(f"   - No duplicate IDs")

print("\n2. DATA QUALITY ISSUES TO HANDLE:")
print(f"   - EDUCATION has undocumented value 0 ({(df['EDUCATION']==0).sum()} cases)")
print(f"   - MARRIAGE has undocumented value 0 ({(df['MARRIAGE']==0).sum()} cases)")
print(f"   - PAY_5 and PAY_6 missing status=1 (one month late)")

print("\n3. KEY DISTRIBUTIONS:")
print(f"   - Gender: {(df['SEX']==2).sum():,} female, {(df['SEX']==1).sum():,} male")
print(f"   - Age range: {df['AGE'].min()} to {df['AGE'].max()} years")
print(f"   - Credit limits: NT${df['LIMIT_BAL'].min():,.0f} to NT${df['LIMIT_BAL'].max():,.0f}")

print("\n4. PAYMENT BEHAVIOR (Most Recent Month):")
pay_0_dist = df['PAY_0'].value_counts()
print(f"   - Paid in full: {pay_0_dist.get(-1, 0):,} ({pay_0_dist.get(-1, 0)/len(df)*100:.1f}%)")
print(f"   - Used revolving: {pay_0_dist.get(0, 0):,} ({pay_0_dist.get(0, 0)/len(df)*100:.1f}%)")
print(f"   - Late payments: {df[df['PAY_0']>0]['PAY_0'].count():,} ({df[df['PAY_0']>0]['PAY_0'].count()/len(df)*100:.1f}%)")

EDA FINDINGS SUMMARY

1. DATA SHAPE:
   - 30,000 customers, 24 features
   - No missing values
   - No duplicate IDs

2. DATA QUALITY ISSUES TO HANDLE:
   - EDUCATION has undocumented value 0 (14 cases)
   - MARRIAGE has undocumented value 0 (54 cases)
   - PAY_5 and PAY_6 missing status=1 (one month late)

3. KEY DISTRIBUTIONS:
   - Gender: 18,112 female, 11,888 male
   - Age range: 21 to 79 years
   - Credit limits: NT$10,000 to NT$1,000,000

4. PAYMENT BEHAVIOR (Most Recent Month):
   - Paid in full: 5,686 (19.0%)
   - Used revolving: 14,737 (49.1%)
   - Late payments: 6,818 (22.7%)


In [13]:
# Check what's ACTUALLY in the original CSV file
with open('/Users/rachelberger/Downloads/UCI_Credit_Card.csv', 'r') as f:
    lines = [f.readline().strip() for i in range(5)]
    for line in lines:
        print(line[:100])  # First 100 characters of each line

ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL
1,20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0
2,120000,2,2,2,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000
3,90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000
4,50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000


- data types should be specified explicitly through seeds/schema.yml to upload to BigQuery